# Iteration

In [1]:
import numpy as np

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)

        self.grad = None
        self.gradient_fn = lambda: None
        self.parents = set()

    def gradient(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.gradient()

    def size(self):
        return len(self.data)

In [3]:
class DataLoader:

    def __init__(self):
        self.features = [[28.1, 58.0],
                         [22.5, 72.0],
                         [31.4, 45.0],
                         [19.8, 85.0],
                         [27.6, 63]]
        self.labels = [[165],
                       [95],
                       [210],
                       [70],
                       [155]]

    def __len__(self):  # 3
        return len(self.features)

    def __getitem__(self, index):  # 4
        return (Tensor(self.features[index]),
                Tensor(self.labels[index]))

In [4]:
class Linear:

    def __init__(self, in_size, out_size):
        self.in_size = in_size
        self.out_size = out_size

        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def __call__(self, x: Tensor):
        return self.forward(x)

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad = p.grad * x.data
            self.bias.grad = np.sum(p.grad, axis=0)

        p.gradient_fn = gradient_fn
        p.parents = {self.weight, self.bias}
        return p

    def parameters(self):
        return [self.weight, self.bias]

In [5]:
class MSELoss:

    def __call__(self, p: Tensor, y: Tensor):
        mse = Tensor(((p.data - y.data) ** 2).mean())

        def gradient_fn():
            p.grad = (p.data - y.data) * 2

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

In [6]:
class SGD:

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def backward(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

In [7]:
LEARNING_RATE = 0.00001

In [8]:
dataset = DataLoader()

feature, label = dataset[0]
model = Linear(feature.size(), label.size())
loss = MSELoss()
sgd = SGD(model.parameters(), LEARNING_RATE)

for i in range(len(dataset)):
    feature, label = dataset[i]

    prediction = model(feature)
    print(f'prediction: {prediction.data}')

    error = loss(prediction, label)
    print(f'error: {error.data}')

    error.gradient()
    sgd.backward()
    print(f"weight: {model.weight.data}")
    print(f"bias: {model.bias.data}")

prediction: [43.05]
error: 14871.802500000002
weight: [[0.5685359 0.641462 ]]
bias: [0.002439]
prediction: [58.97976075]
error: 1297.4576356272407
weight: [[0.58474501 0.69333114]]
bias: [0.0031594]
prediction: [49.56405415]
error: 25739.69272117319
weight: [[0.68549878 0.8377235 ]]
bias: [0.00636812]
prediction: [84.78574114]
error: 218.6181411279782
weight: [[0.67964363 0.81258774]]
bias: [0.00607241]
prediction: [69.9572639]
error: 7232.266962624995
weight: [[0.72658722 0.91974158]]
bias: [0.00777326]
